# Chapter 1: Toy Language

This is an xDSL version of the Toy compiler, as described in the 
[MLIR tutorial](https://mlir.llvm.org/docs/Tutorials/Toy/). Toy is a simple
tensor programming language, with pointwise addition and multiplication. It has 
compile-time checking for shapes, and a built-in print function to observe
the results. In this tutorial we'll compile code to RISC-V assembly, so the
tensors will be 32-bit integers. Given that we want to keep things simple, 
the codegen will be limited to tensors of rank <= 2. All `Values` are 
immutable (i.e. every operation returns a newly allocated value),
and deallocation is automatically managed.

Here is a simple Toy program running in a RISC-V emulator, compiled using xDSL.
Try changing the program and observing the output:

In [1]:
from compiler import compile, emulate_riscv

program = """
def main() {
  # Define a variable `a` with shape <2, 3>, initialized with the literal value.
  # The shape is inferred from the supplied literal.
  var a = [[1, 2, 3], [4, 5, 6]];

  # b is identical to a, the literal tensor is implicitly reshaped: defining new
  # variables is the way to reshape tensors (element count must match).
  var b<3, 2> = [1, 2, 3, 4, 5, 6];

  # There is a built-in print instruction to display the contents of the tensor
  print(b);

  # Reshapes are implicit on assignment
  var c<2, 3> = b;

  # There are + and * operators for pointwise addition and multiplication
  var d = a + c;

  print(d);
}
"""

code = compile(program)
emulate_riscv(code)

[CPU] Started running from example.asm:.text at heap (0x100) + 0x478
Program(name=example.asm,sections=set(),base=['.bss', '.data', '.text'])
[[1, 2], [3, 4], [5, 6]]
[[2, 4, 6], [8, 10, 12]]
[CPU] Program exited with code 0


Type checking is statically performed through type inference; the language only
requires type declarations to specify tensor shapes when needed.

The code for the lexer is fairly straightforward; it is all in a single file:
`toy/lexer.py`. The parser can be found in `toy/parser.py`; it is a recursive 
descent parser. If you are not familiar with such a Lexer/Parser, these are very similar 
to the LLVM Kaleidoscope equivalent that are detailed in the first two chapters of the
[LLVM Kaleidoscope Tutorial](https://llvm.org/docs/tutorial/MyFirstLanguageFrontend/LangImpl02.html).